# n = 5: Autoregressive covariance structure 

We try to parameterize covariance Gamma only using two parameters rho and sigma2 as in the AR(1) structure.

$$\mu_i = 5, \rho = 0.9, \sigma^2 = 0.1, n_i = 5$$

$$\Gamma_i = \sigma^2 * \begin{pmatrix} 1 & \rho & \rho^2 & \rho^3 & \rho^4\\ \rho & 1 & \rho & \rho^2 & \rho^3 \\ \rho^2 & \rho & 1 & \rho & \rho^2 \\ \rho^3 & \rho^2 & \rho & 1 & \rho \\ \rho^4 & \rho^3 & \rho^2 & \rho & 1\end{pmatrix} \forall i \in [1, N = 10,000]$$ 

In [10]:
using DataFrames, Random, GLM, GLMCopula, Test
using LinearAlgebra, BenchmarkTools, Revise

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 5
ρ = 0.9
σ2 = 0.1

V = zeros(n, n) # will store the AR(1) structure without sigma2

mean = 5

dist = Poisson

V = get_AR_cov(n, ρ, σ2, V)

# true Gamma
Γ = σ2 * V

5×5 Array{Float64,2}:
 0.1      0.09    0.081  0.0729  0.06561
 0.09     0.1     0.09   0.081   0.0729
 0.081    0.09    0.1    0.09    0.081
 0.0729   0.081   0.09   0.1     0.09
 0.06561  0.0729  0.081  0.09    0.1

In [11]:
vecd = [dist(mean) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Array{Array{Float64,1},1}:
 [3.0, 8.0, 7.0, 4.0, 2.0]
 [2.0, 6.0, 6.0, 6.0, 7.0]
 [3.0, 5.0, 5.0, 10.0, 3.0]
 [5.0, 6.0, 3.0, 5.0, 2.0]
 [1.0, 10.0, 1.0, 2.0, 2.0]
 [5.0, 5.0, 4.0, 5.0, 4.0]
 [1.0, 4.0, 6.0, 5.0, 5.0]
 [7.0, 8.0, 6.0, 4.0, 9.0]
 [5.0, 3.0, 4.0, 6.0, 4.0]
 [5.0, 5.0, 6.0, 4.0, 7.0]
 [3.0, 7.0, 6.0, 3.0, 6.0]
 [6.0, 3.0, 3.0, 8.0, 5.0]
 [3.0, 4.0, 7.0, 3.0, 4.0]
 ⋮
 [7.0, 4.0, 8.0, 5.0, 3.0]
 [2.0, 4.0, 4.0, 7.0, 7.0]
 [7.0, 2.0, 4.0, 6.0, 8.0]
 [7.0, 4.0, 8.0, 2.0, 6.0]
 [3.0, 6.0, 5.0, 5.0, 1.0]
 [3.0, 4.0, 4.0, 7.0, 7.0]
 [3.0, 4.0, 8.0, 2.0, 3.0]
 [2.0, 9.0, 7.0, 6.0, 6.0]
 [6.0, 4.0, 3.0, 4.0, 7.0]
 [6.0, 4.0, 4.0, 9.0, 6.0]
 [2.0, 3.0, 2.0, 5.0, 4.0]
 [4.0, 4.0, 2.0, 4.0, 4.0]

In [12]:
Random.seed!(1234)

d = Poisson()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaARObs{T, D, Link}}(undef, N)

for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    gcs[i] = GLMCopulaARObs(y, X, d, link)
end

gcm = GLMCopulaARModel(gcs);

In [13]:
initialize_model!(gcm)
@show gcm.β
@show exp.(gcm.β);
fill!(gcm.ρ, 0.0)
fill!(gcm.σ2, 0.0)

loglikelihood!(gcm, true, true)

gcm1 = deepcopy(gcm)
gcm2 = deepcopy(gcm)
gcm3 = deepcopy(gcm);

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -112233.55795434186 39999
2 -112233.55795434186 -112233.55795434186 39999
gcm.β = [1.620377852472435]
exp.(gcm.β) = [5.0550000000000015]


# Quasi Newton
## Number of Iterations....: 25
## 0.79 seconds 

In [14]:
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, tol = 10^-6, hessian_approximation = "limited-memory"))

gcm.θ = [1.620377852472435, 0.0, 0.0]
gcm.∇θ = [-1.6888890286281821e-10, 0.0, 1942.1117705242389]
gcm.θ = [1.620377852472435, 0.00999999, 0.00999999]
gcm.∇θ = [-256.4258556825465, 26.074233261376982, 1578.4451108237101]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:      

-112013.0095314712

# Newton
## Number of Iterations....: 22
## 1.1 seconds 

In [15]:
@time GLMCopula.fit!(gcm2, IpoptSolver(print_level = 5, max_iter = 100, tol = 10^-6, hessian_approximation = "exact"))

gcm.θ = [1.620377852472435, 0.0, 0.0]
gcm.∇θ = [-1.6888890286281821e-10, 0.0, 1942.1117705242389]
gcm.θ = [1.620377852472435, 0.00999999, 0.00999999]
gcm.∇θ = [-256.4258556825465, 26.074233261376982, 1578.4451108237101]
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        3
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:      

-112013.00953147106

In [16]:
@show gcm.∇θ
@show gcm2.∇θ;

gcm.∇θ = [-1.9254077319175167e-6, -2.2590426579371936e-7, -2.63449664750226e-7]
gcm2.∇θ = [1.9219187506358537e-5, 1.150223903523262e-5, -1.965728171571257e-5]


In [17]:
@show gcm.θ
@show gcm2.θ;

gcm.θ = [1.6123212634033952, 0.8695969637523606, 0.09778052303210624]
gcm2.θ = [1.6123212630376447, 0.8695969433710478, 0.09778052722983308]


In [18]:
@show loglikelihood!(gcm, true, true)
@show loglikelihood!(gcm2, true, true);

loglikelihood!(gcm, true, true) = -112013.0095314712
loglikelihood!(gcm2, true, true) = -112013.00953147106
